In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append('..')

In [3]:
import copy
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## Model creation

### DGCNN branch

In [4]:
from dgcnn.model import get_graph_feature


class DGCNNFeatureExtractor(nn.Module):
    def __init__(self, opt: dict):
        '''
        opt structure:
            'k': int, for k-NN
            'emb_dims': int, used to calc hidden size for various layers
        '''
        super(DGCNNFeatureExtractor, self).__init__()
        self.k = opt['k']
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(opt['emb_dims'])

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, opt['emb_dims'], kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))

        self.feat_dim = opt['emb_dims'] * 2


    def forward(self, x):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k)
        x = self.conv1(x)
        x1 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x1, k=self.k)
        x = self.conv2(x)
        x2 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x2, k=self.k)
        x = self.conv3(x)
        x3 = x.max(dim=-1, keepdim=False)[0]

        x = get_graph_feature(x3, k=self.k)
        x = self.conv4(x)
        x4 = x.max(dim=-1, keepdim=False)[0]

        x = torch.cat((x1, x2, x3, x4), dim=1)

        x = self.conv5(x)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)
        x = torch.cat((x1, x2), 1)

        return x

In [5]:
dgcnn_opt = {
    'k': 5,
    'emb_dims': 128,
}

dgcnn_feat_ex = DGCNNFeatureExtractor(dgcnn_opt)
dgcnn_feat_ex.eval()
dgcnn_feat_ex, dgcnn_feat_ex(torch.randn((8,3,1024))).shape

(DGCNNFeatureExtractor(
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (bn5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (conv1): Sequential(
     (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): LeakyReLU(negative_slope=0.2)
   )
   (conv2): Sequential(
     (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): LeakyReLU(negative_slope=0.2)
   )
   (conv3): Sequential(
     (0): Conv2d(128, 128, kernel_siz

### MeshCNN branch

In [6]:
from meshcnn.models.networks import MResConv, get_norm_args, get_norm_layer
from meshcnn.models.layers.mesh_pool import MeshPool
from meshcnn.models.layers.mesh import Mesh


class MeshCNNFeatureExtractor(nn.Module):
    def __init__(self, opt: dict):
        '''
        opt structure:
            'nf0': int
                num input channels (5 for the usual MeshCNN initial edge features)
                Corresponds to "opt.input_nc" in original code, with no default (inferred from dataset)

            'conv_res': list of ints
                num out channels (i.e. filters) for each meshconv layer
                Corresponds to "opt.ncf" in original code, with default [16, 32, 32]

            'input_res': int
                num input edges (we take only this many edges from each input mesh)
                Corresponds to "opt.ninput_edges" in original code, with default 750

            'pool_res': list of ints
                num edges to keep after each meshpool layer
                Corresponds to "opt.pool_res" in original code, with default [1140, 780, 580] 

            'norm': str, one of ['batch', 'instance', 'group', 'none']
                type of norm layer to use
                Corresponds to "opt.norm" in original code, with default 'batch'

            'num_groups': int
                num of groups for groupnorm
                Corresponds to "opt.num_groups" in original code, with default 16

            'nresblocks': int
                num res blocks in each mresconv
                Corresponds to "opt.resblocks" in original code, with default 0
        '''
        super(MeshCNNFeatureExtractor, self).__init__()
        self.k = [opt['nf0']] + opt['conv_res']
        self.res = [opt['input_res']] + opt['pool_res']

        norm_layer = get_norm_layer(norm_type=opt['norm'], num_groups=opt['num_groups'])
        norm_args = get_norm_args(norm_layer, self.k[1:])

        for i, ki in enumerate(self.k[:-1]):
            setattr(self, 'conv{}'.format(i), MResConv(ki, self.k[i + 1], opt['nresblocks']))
            setattr(self, 'norm{}'.format(i), norm_layer(**norm_args[i]))
            setattr(self, 'pool{}'.format(i), MeshPool(self.res[i + 1]))


        self.gp = nn.AvgPool1d(self.res[-1])
        # self.gp = nn.MaxPool1d(self.res[-1])

        self.feat_dim = self.k[-1]

    def forward(self, x, mesh):

        for i in range(len(self.k) - 1):
            x = getattr(self, 'conv{}'.format(i))(x, mesh)
            x = F.relu(getattr(self, 'norm{}'.format(i))(x))
            x = getattr(self, 'pool{}'.format(i))(x, mesh)

        x = self.gp(x)
        x = x.view(-1, self.k[-1])

        return x


In [7]:
'''
meshcnn data loader item format - dict, with keys:
    'mesh': Mesh class instance
    'label': Output class label
    'edge_features': Features extracted using extract_features() of the Mesh object above,
                     but PADDED TO ninput_edges AND NORMALIZED BY MEAN&STD OF DATA
'''

mesh = Mesh(
    file=r'C:\Academic\GT - MSCS\Sem II - Spring 2022\CS 7643 - DL\Project\src\TriangleMesh\data\shrec_16\armadillo\test\T55.obj',
    opt=None, export_folder=None)
test_data = {
    'mesh': mesh,
    'label': 0,
    'edge_features': mesh.extract_features(),
}
test_data, test_data['edge_features'].shape

({'mesh': <meshcnn.models.layers.mesh.Mesh at 0x25be3088088>,
  'label': 0,
  'edge_features': array([[1.57387046, 1.57369169, 1.57342915, ..., 1.57413897, 1.57166371,
          1.57090315],
         [1.35346367, 1.31874424, 1.4147537 , ..., 1.28607998, 1.41143283,
          1.52148169],
         [1.46005855, 1.49583822, 1.47602932, ..., 1.35379826, 1.48621636,
          1.53421813],
         [1.01332234, 0.83147991, 0.89266225, ..., 1.33598894, 0.85190091,
          0.65474185],
         [1.46880606, 1.35175648, 1.14274885, ..., 1.6005793 , 1.03006614,
          0.68757185]])},
 (5, 750))

In [8]:
test_data['edge_features'][None,...,None].shape, [test_data['mesh']]

((1, 5, 750, 1), [<meshcnn.models.layers.mesh.Mesh at 0x25be3088088>])

In [9]:
meshcnn_opt = {
    'nf0': test_data['edge_features'].shape[0],
    'conv_res': [64, 128, 256, 256],
    'input_res': test_data['edge_features'].shape[1],
    'pool_res': [600, 450, 300, 180],
    'norm': 'group',
    'num_groups': 16,
    'nresblocks': 1,
}

meshcnn_feat_ex = MeshCNNFeatureExtractor(meshcnn_opt)
meshcnn_feat_ex.eval()
meshcnn_feat_ex, meshcnn_feat_ex(
    torch.from_numpy(test_data['edge_features'][None,...,None]).float().to('cpu'),
    [test_data['mesh']]).shape

(MeshCNNFeatureExtractor(
   (conv0): MResConv(
     (conv0): MeshConv(
       (conv): Conv2d(5, 64, kernel_size=(1, 5), stride=(1, 1), bias=False)
     )
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv1): MeshConv(
       (conv): Conv2d(64, 64, kernel_size=(1, 5), stride=(1, 1), bias=False)
     )
   )
   (norm0): GroupNorm(16, 64, eps=1e-05, affine=True)
   (pool0): MeshPool()
   (conv1): MResConv(
     (conv0): MeshConv(
       (conv): Conv2d(64, 128, kernel_size=(1, 5), stride=(1, 1), bias=False)
     )
     (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv1): MeshConv(
       (conv): Conv2d(128, 128, kernel_size=(1, 5), stride=(1, 1), bias=False)
     )
   )
   (norm1): GroupNorm(16, 128, eps=1e-05, affine=True)
   (pool1): MeshPool()
   (conv2): MResConv(
     (conv0): MeshConv(
       (conv): Conv2d(128, 256, kernel_size=(1, 5), stride=(1, 1), bias=False)
     )
     (bn1): Ba

### DGCNN branch on MeshCNN style data

In [10]:
type(mesh.vs), mesh.vs.dtype, mesh.vs.shape, torch.from_numpy(mesh.vs.T[None,...]).shape

(numpy.ndarray, dtype('float64'), (252, 3), torch.Size([1, 3, 252]))

In [11]:
dgcnn_feat_ex(
    torch.from_numpy(mesh.vs.T[None,...]).float()
).shape

torch.Size([1, 256])

### Combined network (pre-classification)

In [12]:
class CombinedFeatureExtractor(nn.Module):
    def __init__(self, dgcnn_opt: dict, meshcnn_opt: dict):
        super(CombinedFeatureExtractor, self).__init__()

        self.dgcnn_branch = DGCNNFeatureExtractor(dgcnn_opt)
        self.meshcnn_branch = MeshCNNFeatureExtractor(meshcnn_opt)

        self.feat_dim = self.dgcnn_branch.feat_dim + self.meshcnn_branch.feat_dim


    def forward(self, vertex_input_batch, edge_input_batch, mesh_batch):
        vertex_based_feats = self.dgcnn_branch(vertex_input_batch)
        edge_based_feats = self.meshcnn_branch(edge_input_batch, mesh_batch)

        out = torch.cat([vertex_based_feats, edge_based_feats], dim=-1)
        return out

In [13]:
vertex_input_batch = torch.from_numpy(mesh.vs.T[None,...]).float()
edge_input_batch = torch.from_numpy(test_data['edge_features'][None,...,None]).float()
mesh_batch = [test_data['mesh']]

combined_ex = CombinedFeatureExtractor(dgcnn_opt=dgcnn_opt, meshcnn_opt=meshcnn_opt)
combined_ex(vertex_input_batch, edge_input_batch, mesh_batch).shape

torch.Size([1, 512])

### Full network

In [14]:
class CombinedMeshClassifier(nn.Module):
    def __init__(self, opt: dict):
        '''
        opt structure:
            'classifier_opt': dict (nested options), structure:
                'out_block_hidden_dim': hidden layer size of output 2-layer MLP
                'out_num_classes': num classes to classify, i.e. output layer size of output 2-layer MLP
            'dgcnn_opt': dict (nested options), see DGCNN feat ex for details
            'meshcnn_opt': dict (nested options), see MeshCNN feat ex for details
        '''
        super(CombinedMeshClassifier, self).__init__()

        classifier_opt = opt['classifier_opt']

        self.feat_ex = CombinedFeatureExtractor(dgcnn_opt=opt['dgcnn_opt'], meshcnn_opt=opt['meshcnn_opt'])
        self.output_block = nn.Sequential(
            nn.Linear(in_features=self.feat_ex.feat_dim, out_features=classifier_opt['out_block_hidden_dim']),
            nn.ReLU(),
            nn.Linear(in_features=classifier_opt['out_block_hidden_dim'], out_features=classifier_opt['out_num_classes'])
        )

    def forward(self, vertex_input_batch, edge_input_batch, mesh_batch):
        combined_feats = self.feat_ex(vertex_input_batch, edge_input_batch, mesh_batch)
        out = self.output_block(combined_feats)
        return out

In [15]:
classifier_opt = {
    'out_block_hidden_dim': 1024,
    'out_num_classes': 30
}

classifier = CombinedMeshClassifier(
    opt={
        'classifier_opt': classifier_opt,
        'dgcnn_opt': dgcnn_opt,
        'meshcnn_opt': meshcnn_opt,
    }
)

classifier(vertex_input_batch, edge_input_batch, mesh_batch).shape

torch.Size([1, 30])

## Data loading

In [16]:
import pickle
from torch.utils.data import Dataset

from meshcnn.util.util import is_mesh_file, pad

In [17]:
class SHREC16(Dataset):
    def __init__(self, partition, device, opt: dict):
        '''
        opt structure:
            'ninput_edges': int, num edges to use for meshcnn (will pad if higher than actual)
            'num_points': int, num verts to use for dgcnn (has to be at most the actual num verts)
            'dataroot': str
        '''
        super(SHREC16, self).__init__()
        self.partition = partition
        self.device = device

        self.ninput_edges = opt['ninput_edges']
        self.num_points = opt['num_points']
        self.root = opt['dataroot']
        self.dir = os.path.join(self.root)
        self.classes, self.class_to_idx = self.find_classes(self.dir)
        self.paths = self.make_dataset_by_class(self.dir, self.class_to_idx, partition)
        self.nclasses = len(self.classes)
        self.size = len(self.paths)

        self.mean = 0
        self.std = 1
        self.get_mean_std() # init self.mean, self.std, self.ninput_channels

    def __getitem__(self, index):
        path = self.paths[index][0]
        label = self.paths[index][1]
        mesh = Mesh(file=path, opt=None, hold_history=False, export_folder=None)
        pointcloud = mesh.vs[:self.num_points].T
        meta = {'mesh': mesh, 'label': label, 'pointcloud': pointcloud}

        edge_features = mesh.extract_features()
        edge_features = pad(edge_features, self.ninput_edges)
        meta['edge_features'] = (edge_features - self.mean) / self.std
        return meta

    def __len__(self):
        return self.size


    def get_mean_std(self):
        """ Computes Mean and Standard Deviation from Training Data
        If mean/std file doesn't exist, will compute one
        :returns
        mean: N-dimensional mean
        std: N-dimensional standard deviation
        ninput_channels: N
        (here N=5)
        """

        mean_std_cache = os.path.join(self.root, 'mean_std_cache.pkl')
        if not os.path.isfile(mean_std_cache):
            print('computing mean std from train data...')
            mean, std = np.array(0), np.array(0)
            for i, data in enumerate(self):
                if i % 5 == 0:
                    print('{} of {}'.format(i, self.size))
                features = data['edge_features']
                mean = mean + features.mean(axis=1)
                std = std + features.std(axis=1)
            mean = mean / (i + 1)
            std = std / (i + 1)
            transform_dict = {'mean': mean[:, np.newaxis], 'std': std[:, np.newaxis],
                              'ninput_channels': len(mean)}
            with open(mean_std_cache, 'wb') as f:
                pickle.dump(transform_dict, f)
            print('saved: ', mean_std_cache)

        # open mean / std from file
        with open(mean_std_cache, 'rb') as f:
            transform_dict = pickle.load(f)
            print('loaded mean / std from cache')
            self.mean = transform_dict['mean']
            self.std = transform_dict['std']
            self.ninput_channels = transform_dict['ninput_channels']

    # this is when the folders are organized by class...
    @staticmethod
    def find_classes(dir):
        classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    @staticmethod
    def make_dataset_by_class(dir, class_to_idx, partition):
        meshes = []
        dir = os.path.expanduser(dir)
        for target in sorted(os.listdir(dir)):
            d = os.path.join(dir, target)
            if not os.path.isdir(d):
                continue
            for root, _, fnames in sorted(os.walk(d)):
                for fname in sorted(fnames):
                    if is_mesh_file(fname) and (root.count(partition)==1):
                        path = os.path.join(root, fname)
                        item = (path, class_to_idx[target])
                        meshes.append(item)
        return meshes

In [18]:
import functools

def collate_fn(batch, device, is_train):
    """Creates mini-batch tensors
    We should build custom collate_fn rather than using default collate_fn
    """
    meta = {}
    keys = batch[0].keys()
    for key in keys:
        meta.update({key: np.array([d[key] for d in batch])})

    input_edge_features = torch.from_numpy(meta['edge_features']).float()
    pointcloud = torch.from_numpy(meta['pointcloud']).float()
    label = torch.from_numpy(meta['label']).long()
    meta['edge_features'] = input_edge_features.to(device).requires_grad_(is_train)
    meta['pointcloud'] = pointcloud.to(device).requires_grad_(is_train)
    meta['label'] = label.to(device)
    # meta['mesh'] already contains the reqd list of meshes
    return meta


class DataLoader:
    """multi-threaded data loading"""

    def __init__(self, partition, opt: dict):
        '''
        opt structure:
            'gpu_ids': list of ints, or None (for cpu)
            'batch_size': int (default: 16)
            'max_dataset_size': int (default: inf)
            'shuffle': bool. Whether to shuffle or not
            'num_threads': int

            'dataset_opt': dict (i.e. nested options), with structure as mentioned in class SHREC16
        '''
        device = torch.device('cuda:{}'.format(opt['gpu_ids'][0])) if opt['gpu_ids'] else torch.device('cpu')
        self.dataset = SHREC16(partition, device, opt['dataset_opt'])

        self.batch_size = opt['batch_size']
        self.max_dataset_size = opt['max_dataset_size'] if opt['max_dataset_size'] else np.inf
        self.dataloader = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=opt['shuffle'],
            num_workers=opt['num_threads'],
            collate_fn=functools.partial(
                collate_fn,
                device=device, is_train=(partition=='train')
            )
        )

    def __len__(self):
        return min(len(self.dataset), self.max_dataset_size)

    def __iter__(self):
        for i, data in enumerate(self.dataloader):
            if i * self.batch_size >= self.max_dataset_size:
                break
            yield data

In [19]:
data_root_dir = r'C:\Academic\GT - MSCS\Sem II - Spring 2022\CS 7643 - DL\Project\src\TriangleMesh\data\shrec_16'
dataloader_opt = {
    'gpu_ids': None,
    'batch_size': 16,
    'max_dataset_size': np.inf,
    'shuffle': True,
    'num_threads': 0,
    'dataset_opt': {
        'ninput_edges': 750,
        'num_points': 250,
        'dataroot': data_root_dir,
    },
}

train_dataloader = DataLoader('train', dataloader_opt)
test_dataloader = DataLoader('test', dataloader_opt)
len(train_dataloader), len(test_dataloader)

loaded mean / std from cache
loaded mean / std from cache


(480, 120)

In [20]:
for train_data, test_data in zip(train_dataloader, test_dataloader):
    print(train_data['pointcloud'].shape, test_data['edge_features'].shape)

    print(classifier(train_data['pointcloud'], train_data['edge_features'], train_data['mesh']).shape)
    print(classifier(test_data['pointcloud'], test_data['edge_features'], test_data['mesh']).shape)

    break # comment out if you want to run through whole dataset

torch.Size([16, 3, 250]) torch.Size([16, 5, 750])
torch.Size([16, 30])
torch.Size([16, 30])


## Training loop

### Redefining model wrapper

In [21]:
from meshcnn.util.util import print_network
from meshcnn.models import networks

In [22]:
class ClassifierWrapper:
    def __init__(self, is_train, opt: dict):
        '''
        opt structure:
            'gpu_ids': list of ints, or None (for cpu)
            'checkpoints_dir': str, parent folder for storing models (expt name will be a subfolder here)
            'expt_name': str, name of expt; decides where to store models etc
            'continue_train': bool, whether to resume training from a given epoch (if training).
            'which_epoch': str, which epoch to load if testing or resuming training (default: 'latest'; can enter a number)

            'network_opt': dict (nested options) for model architecture. see CombinedMeshClassifier for details

            'net_init_opt': dict (nested options) for weight init, structure:
                'init_type': str, type of initalization to use, among [normal|xavier|kaiming|orthogonal]. 'normal' is default
                'init_gain': float, gain used for normal/xavier/orthogonal inits. 0.02 is default

            'lr_schedule_opt': dict (nexted options) for LR schedule, structure:
                'lr': float, init LR, default 0.0002
                'lr_policy': str, type of schedule, among lambda|step|plateau, default 'lambda'
                'lr_decay_iters': int, decay by gamma every lr_decay_iters iterations (if step policy). default 50
                'lr_decay_gamma': float, the gamma for decay for step policy. default 0.1
                'epoch_count': int, the starting epoch count (if lambda policy). default 1
                'niter': int, num epochs to be at starting LR (if lambda policy). default 100
                'niter_decay': int, num epochs to decay LR linearly to zero (if lambda policy). default 500.
        '''
        self.gpu_ids = opt['gpu_ids']
        self.is_train = is_train

        self.device = torch.device('cuda:{}'.format(self.gpu_ids[0])) if self.gpu_ids else torch.device('cpu')
        self.save_dir = os.path.join(opt['checkpoints_dir'], opt['expt_name'])
        os.makedirs(self.save_dir, exist_ok=True)
        self.optimizer = None

        # load/define networks
        self.net = CombinedMeshClassifier(opt['network_opt'])
        init_opt = opt['net_init_opt']
        networks.init_net(self.net, init_opt['init_type'], init_opt['init_gain'], self.gpu_ids)
        self.net.train(self.is_train)

        self.criterion = torch.nn.CrossEntropyLoss().to(self.device)
        self.loss = None

        if self.is_train:
            lr_schedule_opt = opt['lr_schedule_opt']
            self.optimizer = torch.optim.Adam(self.net.parameters(), lr=lr_schedule_opt['lr'], betas=(0.9, 0.999))
            self.scheduler = networks.get_scheduler(self.optimizer, lr_schedule_opt)
            print_network(self.net)

        if not self.is_train or opt['continue_train']:
            self.load_network(opt['which_epoch'])

    def forward(self, data_batch):
        out = self.net(data_batch['pointcloud'], data_batch['edge_features'], data_batch['mesh'])
        return out

    def backward(self, out, data_batch):
        self.loss = self.criterion(out, data_batch['label'])
        self.loss.backward()

    def optimize_parameters(self, data_batch):
        self.optimizer.zero_grad()
        out = self.forward(data_batch)
        self.backward(out, data_batch)
        self.optimizer.step()


##################

    def load_network(self, which_epoch):
        """load model from disk"""
        save_filename = '%s_net.pth' % which_epoch
        load_path = os.path.join(self.save_dir, save_filename)
        if not os.path.exists(load_path):
            return  # do nothing; let the net be as it is

        net = self.net
        if isinstance(net, torch.nn.DataParallel):
            net = net.module
        print('loading the model from %s' % load_path)
        state_dict = torch.load(load_path, map_location=self.device)
        if hasattr(state_dict, '_metadata'):
            del state_dict._metadata
        net.load_state_dict(state_dict)

    def save_network(self, which_epoch):
        """save model to disk"""
        save_filename = '%s_net.pth' % (which_epoch)
        save_path = os.path.join(self.save_dir, save_filename)
        if self.gpu_ids and len(self.gpu_ids) > 0 and torch.cuda.is_available():
            torch.save(self.net.module.cpu().state_dict(), save_path)
            self.net.cuda(self.gpu_ids[0])
        else:
            torch.save(self.net.cpu().state_dict(), save_path)

    def update_learning_rate(self):
        """update learning rate (called once every epoch)"""
        self.scheduler.step()
        lr = self.optimizer.param_groups[0]['lr']
        print('learning rate = %.7f' % lr)

    def test(self, data_batch):
        """tests model
        returns: number correct and total number
        """
        with torch.no_grad():
            out = self.forward(data_batch)
            # compute number of correct
            pred_class = out.data.max(1)[1]
            label_class = data_batch['label']
            correct = self.get_accuracy(pred_class, label_class)
        return correct, len(label_class)

    def get_accuracy(self, pred, labels):
        """computes accuracy for classification"""
        correct = pred.eq(labels).sum()
        return correct

### YAML based configs

In [23]:
import yaml

def read_yaml_config(yaml_config_path):
  if yaml_config_path is None:
    return None
  with open(yaml_config_path, 'r') as yml_file:
    cfg = yaml.load(yml_file, Loader=yaml.FullLoader)

  # resolve paths
  def resolve_paths(obj):
    if isinstance(obj, str):
      return (
        os.path.abspath(os.path.join(
          os.path.dirname(yaml_config_path), obj
        ))
        if ('/' in obj or '\\' in obj)
        else obj
      )
    elif isinstance(obj, dict):
      for k,v in obj.items():
        obj[k] = resolve_paths(v)
      return obj
    elif hasattr(obj, '__iter__'):
      for i, x in enumerate(obj):
        obj[i] = resolve_paths(x)
      return obj
    else:
      return obj

  cfg = resolve_paths(cfg)
  return cfg

In [24]:
opt_yaml_path = r'C:\Academic\GT - MSCS\Sem II - Spring 2022\CS 7643 - DL\Project\src\TriangleMesh\configs\combine_mesh_dg\pilot.yml'

opt_full = read_yaml_config(opt_yaml_path)
opt_full

{'dataloader_opt': {'gpu_ids': None,
  'batch_size': 16,
  'max_dataset_size': None,
  'shuffle': True,
  'num_threads': 0,
  'dataset_opt': {'ninput_edges': 750,
   'num_points': 250,
   'dataroot': 'C:\\Academic\\GT - MSCS\\Sem II - Spring 2022\\CS 7643 - DL\\Project\\src\\TriangleMesh\\data\\shrec_16'}},
 'recording_opt': {'print_freq': 10,
  'save_latest_freq': 250,
  'save_epoch_freq': 1,
  'run_test_freq': 1},
 'model_wrapper_opt': {'gpu_ids': None,
  'checkpoints_dir': 'C:\\Academic\\GT - MSCS\\Sem II - Spring 2022\\CS 7643 - DL\\Project\\src\\TriangleMesh\\outputs\\checkpoints',
  'expt_name': 'combine_mesh_dg--pilot',
  'continue_train': False,
  'which_epoch': 'latest',
  'network_opt': {'dgcnn_opt': {'k': 5, 'emb_dims': 128},
   'meshcnn_opt': {'nf0': 5,
    'conv_res': [64, 128, 256, 256],
    'input_res': 750,
    'pool_res': [600, 450, 300, 180],
    'norm': 'group',
    'num_groups': 16,
    'nresblocks': 1},
   'classifier_opt': {'out_block_hidden_dim': 1024, 'out_num_c

### Test (to start with)

In [25]:
from meshcnn.util.writer import Writer

def test(opt: dict, epoch=-1):
    print('Running Test')

    dataloader_opt = opt['dataloader_opt']
    model_wrapper_opt = opt['model_wrapper_opt']

    dataloader = DataLoader('test', dataloader_opt)
    print(f"data num classes: {dataloader.dataset.nclasses}")
    model = ClassifierWrapper(False, model_wrapper_opt)
    writer = Writer(False, model_wrapper_opt['checkpoints_dir'], model_wrapper_opt['expt_name'])
    # test
    writer.reset_counter()
    for i, data in enumerate(dataloader):
        ncorrect, nexamples = model.test(data)
        writer.update_counter(ncorrect, nexamples)
    writer.print_acc(epoch, writer.acc)
    return writer.acc

In [26]:
test(opt_full)

Running Test
loaded mean / std from cache
data num classes: 30
loading the model from C:\Academic\GT - MSCS\Sem II - Spring 2022\CS 7643 - DL\Project\src\TriangleMesh\outputs\checkpoints\combine_mesh_dg--pilot\latest_net.pth
epoch: -1, TEST ACC: [4.1667 %]



0.041666666666666664

### Train

In [29]:
import time


def train(opt: dict):
    dataloader_opt = opt['dataloader_opt']
    model_wrapper_opt = opt['model_wrapper_opt']
    recording_opt = opt['recording_opt']

    dataloader = DataLoader('train', dataloader_opt)
    dataset_size = len(dataloader)
    print('#training meshes = %d' % dataset_size)

    model = ClassifierWrapper(True, model_wrapper_opt)
    writer = Writer(True, model_wrapper_opt['checkpoints_dir'], model_wrapper_opt['expt_name'])

    lr_schedule_opt = model_wrapper_opt['lr_schedule_opt']
    batch_size = dataloader_opt['batch_size']

    total_steps = 0
    for epoch in range(lr_schedule_opt['epoch_count'], lr_schedule_opt['niter'] + lr_schedule_opt['niter_decay'] + 1):
        epoch_start_time = time.time()
        iter_data_time = time.time()
        epoch_iter = 0

        for i, data in enumerate(dataloader):
            iter_start_time = time.time()
            if total_steps % recording_opt['print_freq'] == 0:
                t_data = iter_start_time - iter_data_time
            total_steps += batch_size
            epoch_iter += batch_size
            model.optimize_parameters(data)

            if total_steps % recording_opt['print_freq'] == 0:
                loss = model.loss
                t = (time.time() - iter_start_time) / batch_size
                writer.print_current_losses(epoch, epoch_iter, loss, t, t_data)
                writer.plot_loss(loss, epoch, epoch_iter, dataset_size)

            if i % recording_opt['save_latest_freq'] == 0:
                print('saving the latest model (epoch %d, total_steps %d)' %
                        (epoch, total_steps))
                model.save_network('latest')

            iter_data_time = time.time()
        if epoch % recording_opt['save_epoch_freq'] == 0:
            print('saving the model at the end of epoch %d, iters %d' %
                    (epoch, total_steps))
            model.save_network('latest')
            model.save_network(epoch)

        print('End of epoch %d / %d \t Time Taken: %d sec' %
                (epoch, lr_schedule_opt['niter'] + lr_schedule_opt['niter_decay'], time.time() - epoch_start_time))
        model.update_learning_rate()

        if epoch % recording_opt['run_test_freq'] == 0:
            acc = test(epoch=epoch, opt=opt)
            writer.plot_acc(acc, epoch)

    writer.close()

In [ ]:
train(opt_full)